In [47]:
#IPython magic to reload libraries
%load_ext autoreload
%autoreload 2

# Extra code so Juypter doesn't crash on mac os
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
import sys
sys.path.insert(0, 'utils')  # noqa

import numpy as np
import gym
import ntm
import interfaces

In [49]:
# Training Params
train_iter = 1
batch_size = 1


In [50]:
# Task
env = gym.make('Copy-v0')
env = env.unwrapped
print(env.action_space)
print(env.observation_space)


Tuple(Discrete(2), Discrete(2), Discrete(5))
Discrete(6)


In [85]:
# Controller 
Controller = ntm.NTM()

Model: "model_22"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
tape_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
memory_input (InputLayer)       [(None, 10)]         0                                            
__________________________________________________________________________________________________
concatenate_27 (Concatenate)    (None, 11)           0           tape_input[0][0]                 
                                                                 memory_input[0][0]               
__________________________________________________________________________________________________
dense_47 (Dense)                (None, 32)           384         concatenate_27[0][0]      

In [ ]:
Controller.run(tape_input=1, output_target=1)

tf.Tensor([[3.6993637e-03 9.9205011e-01 8.7449263e-04 1.4536545e-03 1.9223011e-03]], shape=(1, 5), dtype=float32)
Loss:  [0.0079816]


In [21]:
#Define interfaces



In [24]:
memory_content = np.zeros((10,1))

for i in range(train_iter):
    
    # Collect experince using current policy/controller
    for j in range(batch_size):
        episode_steps = 0
        observation = env.reset()
        env.render()

        while True:
            episode_steps += 1
            
            inputs = (observation, memory_content)
            
            Controller.input(inputs)
            Controller.run()
            outputs = Controller.random_output()
            
            input_control = outputs[0] # Direction to move the read head ["left","right"] = [0,1]
            memory_control = outputs[1]
            memory_content = outputs[2]
            output_control = outputs[3] #  Whether to write to the output tape ["no","yes"] = [0,1]
            output_content = outputs[4] # Which character to write (ignored if the above sub-action is 0) 

            actions = (input_control, output_control, output_content)
            observation, reward, done, info = env.step(actions)
            env.render()

            if done:
                done = False
                break
        

Total length of input instance: 3, step: 0
Observation Tape    :   DDE  
Output Tape         :   
Targets             :   DDE  






Total length of input instance: 3, step: 1
Observation Tape    :   DDE  
Output Tape         :   
Targets             :   DDE  

Current reward      :   0.000
Cumulative reward   :   0.000
Action              :   Tuple(move over input: right,
                              write to the output tape: False,
                              prediction: E)


AssertionError: 